In [30]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pendulum
df = pd.read_sql_table('tbl_crimes', 'sqlite:///gdo.db',
                       columns =['NREDS', 'COD_DIAO', 'COD_DIAO1', 'COD_DIAO2','COD_DIAO3','T_C', 'DATA_FATO', 'MUNICIPIO','CIA', 'UEOP', 'QTDE'])

In [15]:
pendulum.now().subtract(years=1, days=1).strftime('%Y-%m-%d')

'2019-07-23'

In [33]:
### FILTRO PARA RETORNAR  APENAS  AS VITIMAS DE  HOMICIDIO CONSUMADO CONFORME CRTITERIOS GDO ###
date_init_before = pendulum.now().subtract(years=1, months=(datetime.today() - timedelta(days=1)).month-1, days=(datetime.today() - timedelta(days=1)).day).strftime('%Y-%m-%d')
date_ende_before = pendulum.now().subtract(years=1, days=1).strftime('%Y-%m-%d')

date_init_actual = pendulum.now().subtract(months=(datetime.today() - timedelta(days=1)).month-1, days=(datetime.today() - timedelta(days=1)).day).strftime('%Y-%m-%d')
date_ende_actual = pendulum.now().subtract(days=1).strftime('%Y-%m-%d')

#date_ende_actual = (datetime.today()- timedelta(days=1)).strftime('%Y-%m-%d') # Infome da  data final da pesquisa

#datetime.strptime(date_init, '%Y-%m-%d')

#datetime.strptime(date_end, '%Y-%m-%d')

df.loc[:,'MES'] = df.loc[:,'DATA_FATO'].dt.month

df.loc[:,'ANO'] = df.loc[:,'DATA_FATO'].dt.year

df_cv = df.loc[
    (
        (df['COD_DIAO'] == 'C01157' )   | (df['COD_DIAO1'] == 'C01157' ) | (df['COD_DIAO2'] == 'C01157' )| (df['COD_DIAO3'] == 'C01157' )
    )
    
                                        & 
    (
        
    (df['DATA_FATO']>=date_init_before) & (df['DATA_FATO']<=date_ende_before)
    
                                        |
        
    (df['DATA_FATO']>=date_init_actual) & (df['DATA_FATO']<=date_ende_actual)
        
    )
]




df_cv

,NREDS,COD_DIAO,COD_DIAO1,COD_DIAO2,COD_DIAO3,T_C,DATA_FATO,MUNICIPIO,CIA,UEOP,QTDE,MES,ANO
3651,2019-000095792-001,C01157,7777777,7777777,7777777,CONSUMADO,2019-01-01,NOVA SERRANA,279ª CIA,60º BPM,1,1,2019
3666,2019-000245049-001,C01157,7777777,7777777,7777777,CONSUMADO,2019-01-02,NOVA SERRANA,280ª CIA,60º BPM,1,1,2019
3667,2019-000253230-001,C01157,7777777,7777777,7777777,CONSUMADO,2019-01-02,ITAUNA,51ª CIA,23º BPM,1,1,2019
3677,2019-000307616-001,C01157,7777777,7777777,7777777,CONSUMADO,2019-01-03,PARA DE MINAS,19ª CIA PM IND,19ª CIA PM IND,1,1,2019
3681,2019-000362187-001,C01157,7777777,7777777,7777777,CONSUMADO,2019-01-03,SAO JOSE DA VARGINHA,19ª CIA PM IND,19ª CIA PM IND,1,1,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51862,2020-014975322-001,C01157,7777777,7777777,7777777,CONSUMADO,2020-03-26,CORREGO DANTA,107ª CIA,07º BPM,1,3,2020
51883,2020-013182420-001,C01157,7777777,7777777,7777777,CONSUMADO,2020-03-14,BAMBUI,241ª CIA,63º BPM,1,3,2020
51900,2019-030213232-001,C01157,7777777,7777777,7777777,CONSUMADO,2019-06-26,TAPIRAI,241ª CIA,63º BPM,1,6,2019
51905,2019-001485157-001,C01157,7777777,7777777,7777777,CONSUMADO,2019-01-10,TAPIRAI,241ª CIA,63º BPM,1,1,2019


In [34]:
df_crv = pd.pivot_table(df_cv,
               index=['UEOP','MUNICIPIO'], 
               values=['QTDE'], 
               #columns=['ANO', 'MES'],
               columns=['ANO'],
               #columns=['MES'],
               fill_value=0,
               aggfunc=np.sum, 
               margins=True, 
               margins_name='Total Geral'
               #observed =       

              )
try:
    
    df_crv['VAR']  = (df_crv[('QTDE',    2020)] *100 / df_crv[('QTDE',    2019)] - 100).round(1)
        
except ZeroDivisionError:
    
    df_crv['VAR'] = 0

df_crv.replace([np.inf, -np.inf], 100, inplace=True)

#df_crv['VAR'].astype(float)


df_crv



QTDE                      VAR
ANO                                    2019  2020 Total Geral       
UEOP           MUNICIPIO                                            
07º BPM        ABAETE                    15    10          25  -33.3
               BIQUINHAS                  1     3           4  200.0
               BOM DESPACHO              55    34          89  -38.2
               CORREGO DANTA              0     5           5  100.0
               DORES DO INDAIA            2     6           8  200.0
               JAPARAIBA                  6     1           7  -83.3
               LAGOA DA PRATA            65    55         120  -15.4
               LUZ                       15    11          26  -26.7
               MARTINHO CAMPOS           14     7          21  -50.0
               MOEMA                      7     5          12  -28.6
               MORADA NOVA DE MINAS       5     0           5 -100.0
               PAINEIRAS                  0     1           1  100.0
               PEDRA DO INDAIA            8     7          15  -12.5
               POMPEU                    34    33          67   -2.9
               QUARTEL GERAL              1     0           1 -100.0
               SANTO ANTONIO DO MONTE    42    19          61  -54.8
19ª CIA PM IND IGARATINGA                11     7          18  -36.4
               MARAVILHAS                 2     4           6  100.0
               ONCA DO PITANGUI           3     1           4  -66.7
               PAPAGAIOS                 12     2          14  -83.3
               PARA DE MINAS            118    75         193  -36.4
               PEQUI                      2     0           2 -100.0
               SAO JOSE DA VARGINHA       6     3           9  -50.0
23º BPM        CARMO DO CAJURU           24    17          41  -29.2
               CLAUDIO                   30    27          57  -10.0
               DIVINOPOLIS              609   333         942  -45.3
               ITATIAIUCU                12    14          26   16.7
               ITAUNA                   144    81         225  -43.8
               SAO GONCALO DO PARA       26    22          48  -15.4
60º BPM        ARAUJOS                   14     6          20  -57.1
               CONCEICAO DO PARA         13    11          24  -15.4
               LEANDRO FERREIRA           2     3           5   50.0
               NOVA SERRANA             549   342         891  -37.7
               PERDIGAO                  43    27          70  -37.2
               PITANGUI                  61    34          95  -44.3
63º BPM        ARCOS                      9     3          12  -66.7
               BAMBUI                    13     7          20  -46.2
               CORREGO FUNDO              1     2           3  100.0
               FORMIGA                   41    27          68  -34.1
               IGUATAMA                   4     0           4 -100.0
               ITAPECERICA               32    12          44  -62.5
               MEDEIROS                   1     0           1 -100.0
               PAINS                      3     0           3 -100.0
               PIMENTA                    4     2           6  -50.0
               SAO SEBASTIAO DO OESTE     9     6          15  -33.3
               TAPIRAI                    3     1           4  -66.7
Total Geral                            2071  1266        3337  -38.9

In [10]:
def farol_colors(val):
    feliz = '&#128578'
    normal = '&#x1f610'
    triste = '&#128577'
    if val < 0 :
        color = 'green'
    elif val == 0:
        color = 'yellow'
    elif val > 0:
        color = 'red'
    else:
        color = 'orange'
    return (
        '''                
        background-color: {};
        font-size: 14px;
        text-align: center;        
        '''.format(color)
    )

df_hcc_color = df_hcc.style.applymap(farol_colors, subset=['VAR'])



from IPython.display import display, HTML
title = display(HTML(' <span style="color:orange"><font size=0pt><h1>VITIMAS DE HOMICIDIOS CONSUMADOS</h1> </span>  ')
       
       )

html = df_hcc_color.render()

import imgkit
path_wkthmltoimage = r'C:\Program Files\wkhtmltopdf\bin\wkhtmltoimage.exe'
config = imgkit.config(wkhtmltoimage=path_wkthmltoimage)


imgkit.from_string(html, 'Vitímas de Homicídio Consumado.jpg',config=config)





Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


True

In [11]:
df_hcc.style.set_table_styles( 
[ 
 {'selector': 'th', 
  'props': [('background', '# 606060'),  
            ('color', 'green'), ]}, 
 {'selector': 'td', 
  'props': [('color', 'blue')]}, 
]) 
